In [1]:
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings('ignore')
from sqlalchemy import create_engine, text, inspect

# Database credentials
db_host = 'LBHHLWSQL0001.lbhf.gov.uk'
db_port = '1433'
db_name = 'IA_ODS'

# Create the connection string for SQL Server using pyodbc with Windows Authentication
connection_string = f'mssql+pyodbc://@{db_host}:{db_port}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes'

# Create the database engine
engine = create_engine(connection_string)


# Function for getting the name of a DataFrame
def get_var_name(var):
    try:
        for name, value in globals().items():
            if value is var:
                return name
    except Exception as e:
        print(f'Error getting variable name: {e}')
    return None

def validate_data(df, show_counts=True):
    try:
        df_name = get_var_name(df)
        print(f'#########################################################################################################################################################################################\nDataFrame: {df_name}')
        # Snapshot the dataset
        display(df)
        # Check for unique values
        unique_counts = pd.DataFrame(df.nunique())
        unique_counts = unique_counts.reset_index().rename(columns={0:'No. of Unique Values', 'index':'Field Name'})
        print("Unique values per field:")
        pd.set_option('display.max_rows', None)
        display(unique_counts)
        pd.reset_option('display.max_rows')
        # Checking for duplicates
        duplicate_count = df.duplicated().sum()
        print("\nNumber of duplicate rows:")
        print(duplicate_count,'\n')
        info = df.info(show_counts=show_counts)
        display(info)
        # Summary stats
        print("\nSummary statistics:")
        display(df.describe())
        print('End of data validation\n#########################################################################################################################################################################################\n')
    except Exception as e:
        print(f'Error validating data: {e}')

def export_to_csv(df, **kwargs):
    try:
        # Obtaining wanted directory
        directory = kwargs.get('directory',r"C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry")
        
        # Obtaining name of DataFrame
        df_name = kwargs.get('df_name',get_var_name(df))
        if not isinstance(df_name, str) or df_name == '_':
                df_name = input('Dataframe not found in global variables. Please enter a name for the DataFrame: ')

        file_path = f'{directory}\\{df_name}.csv'

        print(f'Exproting {df_name} to CSV...\n@ {file_path}\n')
        df.to_csv(file_path, index=False)
        print(f'Successfully exported {df_name} to CSV')
    except Exception as e:
        print(f'Error exporting to CSV: {e}')

def query_data(schema, data):
    try:
        # Define the SQL query
        query = f'SELECT * FROM [{schema}].[{data}]'
        # Load data into DataFrame
        df = pd.read_sql(query, engine)
        print(f'Successfully imported {data}')
        return df
    except Exception as e:
        print(f'Error querying data: {e}')
        return pd.DataFrame()

def read_directory():
    directory = os.getcwd()
    files = os.listdir(os.getcwd())
    print(f"Your Current Directory is: {directory}")
    print("Files in: %s" % (files))

In [2]:
customer_columns = {
    'InteractionsKey':'interactionskey',
    'ETLTaskID':'etltaskid',
    'RowNumber':'rownumber',
    'GroupName':'groupname',
    'Type':'type',
    'Direction':'direction',
    'TimeQueued':'timequeued',
    'TimeEnded':'timeended',
    'Queue':'queue',
    'Skill':'skill',
    'Agent':'agent',
    'Username':'username',
    'UserID':'userid',
    'CustomerName':'customername',
    'CustomerContact':'customercontact',
    'QueueWait':'queuewait',
    'Duration':'duration',
    'TalkTime':'talktime',
    'CallHoldDuration':'callholdduration',
    'WrapUpDuration':'wrapupduration',
    'AgentHandlingTime':'agenthandlingtime',
    'Result':'result',
    'ActivityCode':'activitycode',
    'AssistedBy':'assistedby',
    'TransferredTo':'transferredto',
    'MediaInteractionId':'interactionid',
    'Subject':'subject',
    'CaseReference':'casereference',
    'Tags':'tags',
    'Forwarded':'forwarded',
    'IPAddress':'ipaddress',
    'FormattedSource':'formattedsource',
    'ConversationID':'conversationid',
    'ConversationStartTime':'conversationstarttime',
    'ConversationEndTime':'conversationendtime',
    'ConversationDuration':'conversationduration'
}
master_columns = {
    'Interaction Id':'interactionid',
    'Type':'type',
    'Direction':'direction',
    'Time Queued':'timequeued',
    'Time Ended':'timeended',
    'Queue':'queue',
    'Skill':'skill',
    'Agent':'agent',
    'Customer Name':'customername',
    'Customer Contact':'customercontact',
    'Queue Time':'queuetime',
    'Duration':'duration',
    'Talk Time':'talktime',
    'Hold Time':'holdtime',
    'Wrap-up Duration':'wrap-upduration',
    'Agent Handling Time':'agenthandlingtime',
    'Result':'result',
    'Activity Code':'activitycode',
    'Assisted By':'assistedby',
    'Transferred To':'transferredto',
    'Subject':'subject',
    'Conversation Start Time':'conversationstarttime',
    'Conversation End Time':'conversationendtime',
    'Conversation Duration':'conversationduration',
    'Agent Username':'agentusername',
    'Agent ID':'agentid',
    'Case Reference':'casereference',
    'Forwarded':'forwarded',
    'Source':'source',
    'Conversation ID':'conversationid',
    'IP Address':'ipaddress'
}

In [3]:
customer_service_data = query_data(schema='Netcall', data='Interactions')
customer_service_data

Error querying data: (pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)


""


In [4]:
root_dir = r"C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Interactions Data"
os.chdir(root_dir)

read_directory()

# Create an empty DataFrame to store the combined data
master_data = pd.DataFrame()
 
# Iterate over each folder in the root directory
for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)
    # Ensure it's a directory
    if os.path.isdir(folder_path):
        # Iterate over each CSV file in the folder
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):  # Ensure it's a CSV file
                file_path = os.path.join(folder_path, file)
 
                # Read CSV
                df = pd.read_csv(file_path)
 
                # Add a new column with the CSV filename
                df["Source_File"] = file
 
                # Append to master DataFrame
                master_data = pd.concat([master_data, df], ignore_index=True)

Your Current Directory is: C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Interactions Data
Files in: ['desktop.ini', 'Interactions Feb 2023 - Apr 2024 (excl Cust Serv)', 'Interactions Jan 2022 - Dec 2022 (excl Cust Serv)', 'Interactions Jan 2024 - Jun 2024 (excl Cust Serv)', 'Interactions Jun 24 - Jan 25 (excl Cust Serv)', 'Interactions_Jan 2023 (LIMITED GROUPS)']


In [5]:
customer_service = customer_service_data.copy()
customer_service.rename(
    columns=customer_columns,
    inplace=True
)
customer_service = customer_service[customer_service['groupname'] != 'Adult_Social_Care']

columns_to_drop = [
    'interactionskey','etltaskid','rownumber',
    'username','userid','queuewait','callholdduration',
    'wrapupduration','tags','formattedsource'
]
customer_service.drop(columns=columns_to_drop, inplace=True)
col = customer_service.pop('interactionid')
customer_service.insert(0, 'interactionid', col)
customer_service['groupname'] = customer_service['groupname'].str.replace('Customer_Services','Customer Services')

customer_service.sort_values(
    ['timequeued'],
    ascending=True,
    inplace=True
)

drop = customer_service[customer_service['interactionid'].duplicated(keep='first')]
customer_service.drop(
    drop.index,
    inplace=True
)
customer_service.reset_index(
    drop='index',
    inplace=True
)

customer_service['timequeued'] = pd.to_datetime(customer_service['timequeued'], format='mixed', dayfirst=True)
customer_service['timeended'] = pd.to_datetime(customer_service['timeended'], format='mixed', dayfirst=True)
customer_service['Year'] = customer_service['timequeued'].dt.year
customer_service['Month'] = customer_service['timequeued'].dt.month

KeyError: 'groupname'

In [ ]:
master_df = master_data.copy()
master_df.rename(
    columns=master_columns,
    inplace=True
)

# Step 1: Split at the first dot (.)
master_df[['groupname', 'to_drop']] = master_df['Source_File'].str.split('.', expand=True)
 
# Step 2: Split at the first opening parenthesis (()
master_df[['groupname', 'to_drop']] = master_df['groupname'].str.split('(', expand=True)
 
# Step 3: Clean text (remove punctuation & normalize spaces)
master_df['groupname'] = master_df['groupname'].apply(lambda text: re.sub(r"[^\w\s]", "", text).strip())
master_df['groupname'] = master_df['groupname'].str.replace('Childrens', 'Children')
master_df['groupname'] = master_df['groupname'].str.replace('Children', 'Childrens')
master_df['groupname'] = master_df['groupname'].str.replace('ICAT6600', 'ICAT 6600')
master_df['groupname'] = master_df['groupname'].str.replace('Refugee', 'Refuge')
master_df['groupname'] = master_df['groupname'].str.replace('Refuge', 'Refugee')
master_df['groupname'] = master_df['groupname'].str.replace('Netcall HMS', 'HMS')
master_df['groupname'] = master_df['groupname'].str.replace('HMS', 'Netcall HMS')
master_df['groupname'] = master_df['groupname'].str.replace('Recovery Queue', 'Recovery')

col = master_df.pop('interactionid')
master_df.insert(0, 'interactionid', col)
col = master_df.pop('groupname')
master_df.insert(1, 'groupname', col)

master_df.sort_values(
    ['timequeued','agentid'],
    ascending=True,
    inplace=True
)

master_df.drop(
    columns=['Tags','Source_File','to_drop','Group'],
    inplace=True
)
drop = master_df[master_df['interactionid'].duplicated(keep='first')]
master_df.drop(
    drop.index,
    inplace=True
)
master_df.reset_index(
    drop='index',
    inplace=True
)


master_df['timequeued'] = pd.to_datetime(master_df['timequeued'], format='mixed', dayfirst=True)
master_df['timeended'] = pd.to_datetime(master_df['timeended'], format='mixed', dayfirst=True)
master_df['Year'] = master_df['timequeued'].dt.year
master_df['Month'] = master_df['timequeued'].dt.month

In [ ]:
pd.set_option('display.max_columns', None)
final_df = pd.concat([master_df,customer_service])

values_to_change = [
    'Unknown','Completed','0'
]
for value in values_to_change:
    if value == '0':
        final_df['result'] = final_df['result'].str.replace('0','Failed')
    else:
        final_df['result'] = final_df['result'].str.replace(f'{value}','Other')

final_df['nullcount'] = final_df.isna().sum(axis=1)
final_df.sort_values(by=['interactionid','nullcount'], inplace=True)
drop = final_df[final_df['interactionid'].duplicated(keep='first')]
final_df.drop(drop.index, inplace=True)
final_df.drop(columns='nullcount', inplace=True)

validate_data(final_df)

In [ ]:
aggregated_interactions = final_df.groupby(['direction','type','Year','Month','groupname','queue']).agg(
    number_of_interactions = ('interactionid', 'count')
).reset_index()

export_to_csv(aggregated_interactions)